In [1]:
from torchkf import * 
import numpy as np
import scipy as sp
import sympy

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
""" format: value[, cov[, constaint]]
If constraint == 'positive', then the cov must be in log-space
""" 
parameters = dotdict(
    skeletal=dotdict(
        weight=dotdict(
            body=(70, np.exp(0), 'positive'), 
            relative=dotdict(
                forearm=(0.00762, np.exp(-4), 'positive'),
                upper_arm=(0.01373, np.exp(-4), 'positive'), 
                hand=(0.00361, np.exp(-4), 'positive'),
            ), 
        ), 
        size=dotdict(
            body=(1.75, np.exp(0), 'positive'), 
            relative=dotdict(
                forearm=(0.16480, np.exp(-4), 'positive'),
                upper_arm=(0.19590, np.exp(-4), 'positive'), 
                hand=(0.11465, np.exp(-4), 'positive'),
                humerus_factor=(0.8, np.exp(-4), 'positive'),
            ), 
        ),
        constraints=dotdict(
            elbow=dotdict(
                max_flexion=np.pi / 6,
                max_extension=9 * np.pi / 10.,
            )
        ),
        g=9.81,
    ),
    muscles=dotdict(
        triceps=dotdict(
            insertion=(0.02, np.exp(-8), 'positive'),
            tendon_length=(0.12, np.exp(-2), 'positive'),
            hill_muscle=dotdict(
                Fmax = (2397.12, np.exp(2), 'positive'), # N
                Lm0  = (0.15, np.exp(-2), 'positive'), # m
                phi0 = (0.2094, np.exp(-2)), # rad
                W    = (0.56, np.exp(-2), 'positive'),
                vmax = (1.3, np.exp(-2), 'positive'), # m/s
                A    = 0.25, 
                gmax = 1.5, 
                Lslack = (0.18, np.exp(0), 'positive'), # m
                Kmee1 = 10, 
                Kmee2 = 10_000,
                Tact  = 0.1,
            ),
        ), 
        biceps=dotdict(
            insertion=(0.02, np.exp(-8), 'positive'),
            tendon_length=(0.12, np.exp(-2), 'positive'),
            hill_muscle=dotdict(
                Fmax = (2874.67, np.exp(2), 'positive'), # N
                Lm0  = (0.13, np.exp(-2), 'positive'), # m
                phi0 = (0, np.exp(-2)), # rad
                W    = (0.56, np.exp(-2), 'positive'),
                vmax = (1.3, np.exp(-2), 'positive'), # m/s
                A    = 0.25, 
                gmax = 1.5, 
                Lslack = (0.21, np.exp(0), 'positive'), # m
                Kmee1 = 10, 
                Kmee2 = 10_000,
                Tact  = 0.1,
            ), 
        ),
    ), 
    afferents=dotdict(
        Ia=dotdict(
            kv=2.5,
            lmf=0.2,
            kdl=0.2,
            knl=0.5, 
            c1= 0.01,
        ), 
        Ib=dotdict(
            kf=1,
            fF=0.1,
        ),
    ),
)

In [3]:
def process_parameters(parameters): 
    proc_params = type(parameters)()
    for key, val in parameters.items(): 
        if isinstance(val, dict): 
            _proc_params = process_parameters(val)
            proc_params[key] = _proc_params
        else: 
            try: n = len(val)
            except TypeError: n = 0
            if n > 3: raise AttributeError(f'Parameter {key} has too many values (expected length 3)')
            param = [0, 0, None]
            
            # parameter value (prior expectation)
            if n == 0: param[0] = val
            else: param[0] = val[0]
                
            # parameter covariance
            if n > 1: param[1] = val[1]
                
            # parameter constraint
            if n == 3: param[2] = val[2]
            
            proc_params[key] = tuple(param)
    return proc_params
            
def flatten_parameters(parameters, namespace=''):
    flat_params = type(parameters)()
    for key, val in parameters.items(): 
        nkey = f'{namespace}.{key}' if len(namespace) > 0 else key
        if isinstance(val, dict): 
            _flat_params = flatten_parameters(val, namespace=nkey)
            flat_params.update(_flat_params)
        else: 
            flat_params[nkey] = val
    return flat_params            

In [4]:
import pandas as pd
df = pd.DataFrame(flatten_parameters(process_parameters(parameters))).transpose()
df = df.set_axis(['expectation', 'covariance', 'constraint'], axis=1)
df['constraint'] = df['constraint'].apply(lambda x: x if isinstance(x, str) else 'none')
df['covariance'] = df['covariance'].apply(lambda x: f'exp({int(np.log(x))})' if x > 0 else 0)
df

,expectation,covariance,constraint
skeletal.weight.body,70,exp(0),positive
skeletal.weight.relative.forearm,0.00762,exp(-4),positive
skeletal.weight.relative.upper_arm,0.01373,exp(-4),positive
skeletal.weight.relative.hand,0.00361,exp(-4),positive
skeletal.size.body,1.75,exp(0),positive
skeletal.size.relative.forearm,0.1648,exp(-4),positive
skeletal.size.relative.upper_arm,0.1959,exp(-4),positive
skeletal.size.relative.hand,0.11465,exp(-4),positive
skeletal.size.relative.humerus_factor,0.8,exp(-4),positive
skeletal.constraints.elbow.max_flexion,0.523599,0,none


In [5]:
from abc import ABC, abstractmethod

class NamedParamModel(ABC): 
    def __init__(self, 
                 state_names, input_names, param_names, 
                 state_map=None, input_map=None, param_map=None,
                 state_keys=None, input_keys=None, param_keys=None):
        
        self.state_names = state_names
        self.input_names = input_names
        self.param_names = param_names
        
        if state_map is None: 
            self.state_map = {k:k for k in self.state_names}
        else: 
            self.state_map = {
                k:state_map[k] if k in state_map.keys() else k 
                for k in state_names}
        
        if input_map is None: 
            self.input_map = {k:k for k in self.input_names}
        else: 
            self.input_map = {
                k:input_map[k] if k in input_map.keys() else k 
                for k in input_names}
            
        if param_map is None: 
            self.param_map = {k:k for k in self.param_names}
        else: 
            self.param_map = {
                k:param_map[k] if k in param_map.keys() else k 
                for k in param_names}
        
        if None not in (state_keys, input_keys, param_keys): 
            self.compute_indices(state_keys, input_keys, param_keys)
        
    def compute_indices(self, state_keys, input_keys, param_keys):
        self.state_keys = state_keys
        self.input_keys = input_keys
        self.param_keys = param_keys
        
        self.ix = [state_keys.index(self.state_map[k]) for k in self.state_names]
        self.iv = [input_keys.index(self.input_map[k]) for k in self.input_names]
        self.ip = [param_keys.index(self.param_map[k]) for k in self.param_names]
        
    @staticmethod
    def unvec_args(func): 
        from functools import wraps
        @wraps(func)
        def _wrap(self, x, v, p, **kwargs):
            try: 
                _x = x[:, 0][self.ix]
                _v = v[:, 0][self.iv]
                _p = p[:, 0][self.ip]
            except TypeError: 
                _x, _v, _p = x, v, p
            
            return func(self, *_x, *_v, *_p, **kwargs)
        return _wrap

In [6]:
class HillMuscle(NamedParamModel):  
    state_names = 'L', 'vel'
    input_names = 'aMN', 'gMN' 
    param_names = 'Kmee1', 'Kmee2', 'Lslack', 'Fmax', 'Lm0', 'W', 'vmax', 'A', 'gmax', 'phi0', 'kv', 'kdl', 'knl', 'c1', 'kf'
        
    def __init__(self, *args, **kwargs): 
        super().__init__(HillMuscle.state_names, HillMuscle.input_names, HillMuscle.param_names, *args, **kwargs)
    
    @staticmethod
    def force_velocity(vel, vmax, A, gmax): 
        cd    = vmax * A * (gmax - 1.) / (A + 1.)
        expr1 = (vmax + vel) / (vmax - vel/A)
        expr2 = (gmax * vel + cd) / (vel + cd)
        
        return sympy.Piecewise((expr1, vel <= 0), (expr2, True))
#         return expr1
    
    @staticmethod
    def force_length(L, Lm0, W): 
        return sympy.exp(- (L - Lm0)**2/(W * Lm0)**2)
    
    @staticmethod
    def Fmee(L, Kmee1, Kmee2, Lslack):
        expr1 = Kmee1 * (L - Lslack)
        expr2 = Kmee2 * (L - Lslack)**2
        
        return expr1 + sympy.Piecewise((0, (L < Lslack)), (expr2, True))
    
    @staticmethod
    def Fmce(aMN, Fmax, L, vel, Lm0, W, vmax, A, gmax): 
        fl = HillMuscle.force_length(L, Lm0, W)
        fv = HillMuscle.force_velocity(vel, vmax, A, gmax)
        
        return aMN * Fmax * fl * fv
    
    @staticmethod
    def Frte(L, vel, aMN, Kmee1, Kmee2, Lslack, Fmax, Lm0, W, vmax, A, gmax, phi0):     
        Fmee = HillMuscle.Fmee(L, Kmee1, Kmee2, Lslack)
        Fmce = HillMuscle.Fmce(aMN, Fmax, L, vel, Lm0, W, vmax, A, gmax)
        
        return (Fmee + Fmce) * sympy.cos(Lm0 * sympy.sin(phi0) / L)

    @staticmethod
    def rateIa(L, vel, gMN, Lm0, vmax, kv, kdl, knl, c1):
        dnorm = (L - 0.2 * Lm0) / Lm0
        vnorm = vel / vmax
        Ia    = kv * vnorm + kdl * dnorm + knl * gMN + c1 
        Ia    = sympy.Piecewise((0, Ia < 0),  (Ia, (Ia >= 0) & (Ia <= 1)), (1, True))
        return Ia
    
    @staticmethod
    def rateIb(Frte, Fmax, kf): 
        Fnorm = (Frte - 0.1 * Fmax) / Fmax
        Ib    = kf * Fnorm
        
        Ib    = sympy.Piecewise((0, Ib < 0), (Ib, (Ib >=0) & (Ib <=1)), (1, True))
        return Ib
    
    @NamedParamModel.unvec_args
    def compute_output(self, L, vel, aMN, gMN, Kmee1, Kmee2, Lslack, Fmax, Lm0, W, 
                       vmax, A, gmax, phi0, kv, kdl, knl, c1, kf, return_force=True, return_rates=True):      
        y = []
        
        Frte   = HillMuscle.Frte(L, vel, aMN, Kmee1, Kmee2, Lslack, Fmax, W, vmax, A, gmax, Lm0, phi0)
        if return_force: 
            y.append(Frte)
            
        if return_rates: 
            rateIa = HillMuscle.rateIa(L, vel, gMN, Lm0, vmax, kv, kdl, knl, c1)
            rateIb = HillMuscle.rateIb(Frte, Fmax, kf)       
            
            y.extend([rateIa, rateIb])
        if len(y) == 1:
            y, = y
        
        return y

In [7]:
flatten_parameters(process_parameters(parameters))

{'skeletal.weight.body': (70, 1.0, 'positive'),
 'skeletal.weight.relative.forearm': (0.00762,
  0.01831563888873418,
  'positive'),
 'skeletal.weight.relative.upper_arm': (0.01373,
  0.01831563888873418,
  'positive'),
 'skeletal.weight.relative.hand': (0.00361, 0.01831563888873418, 'positive'),
 'skeletal.size.body': (1.75, 1.0, 'positive'),
 'skeletal.size.relative.forearm': (0.1648, 0.01831563888873418, 'positive'),
 'skeletal.size.relative.upper_arm': (0.1959, 0.01831563888873418, 'positive'),
 'skeletal.size.relative.hand': (0.11465, 0.01831563888873418, 'positive'),
 'skeletal.size.relative.humerus_factor': (0.8,
  0.01831563888873418,
  'positive'),
 'skeletal.constraints.elbow.max_flexion': (0.5235987755982988, 0, None),
 'skeletal.constraints.elbow.max_extension': (2.827433388230814, 0, None),
 'skeletal.g': (9.81, 0, None),
 'muscles.triceps.insertion': (0.02, 0.00033546262790251185, 'positive'),
 'muscles.triceps.tendon_length': (0.12, 0.1353352832366127, 'positive'),
 'mus

In [8]:
state_keys = 'L', 'vel'
input_keys = 'aMN', 'gMN'
params     = flatten_parameters(process_parameters(parameters))
param_keys = [*params.keys()]
param_map  = {k: f'muscles.triceps.hill_muscle.{k}' 
              for k in HillMuscle.param_names if f'muscles.triceps.hill_muscle.{k}' in param_keys}
param_map.update(
    {k: f'afferents.Ia.{k}' for k in HillMuscle.param_names if f'afferents.Ia.{k}' in param_keys})
param_map.update(
    {k: f'afferents.Ib.{k}' for k in HillMuscle.param_names if f'afferents.Ib.{k}' in param_keys})

triceps_muscle = HillMuscle(param_map=param_map ,state_keys=state_keys, input_keys=input_keys, param_keys=param_keys)

In [9]:
import pandas as pd
df = pd.DataFrame(flatten_parameters(process_parameters(parameters))).transpose()
df = df.set_axis(['expectation', 'covariance', 'constraint'], axis=1)
df['constraint'] = df['constraint'].apply(lambda x: x if isinstance(x, str) else 'none')
df['covariance'] = df['covariance'].apply(lambda x: f'exp({int(np.log(x))})' if x > 0 else 0)
df

,expectation,covariance,constraint
skeletal.weight.body,70,exp(0),positive
skeletal.weight.relative.forearm,0.00762,exp(-4),positive
skeletal.weight.relative.upper_arm,0.01373,exp(-4),positive
skeletal.weight.relative.hand,0.00361,exp(-4),positive
skeletal.size.body,1.75,exp(0),positive
skeletal.size.relative.forearm,0.1648,exp(-4),positive
skeletal.size.relative.upper_arm,0.1959,exp(-4),positive
skeletal.size.relative.hand,0.11465,exp(-4),positive
skeletal.size.relative.humerus_factor,0.8,exp(-4),positive
skeletal.constraints.elbow.max_flexion,0.523599,0,none


In [10]:
from collections import OrderedDict

In [11]:
class ElbowModel(NamedParamModel):
    state_names = 'q', 'dq'
    input_names = 'q2', 'aMNb', 'aMNt', 'gMNb', 'gMNt'
    param_map   = OrderedDict({
        'uarm_l_rel':    'skeletal.size.relative.upper_arm', 
        'farm_l_rel':    'skeletal.size.relative.forearm', 
        'hand_l_rel':    'skeletal.size.relative.hand', 
        'uarm_w_rel':    'skeletal.weight.relative.upper_arm',
        'farm_w_rel':    'skeletal.weight.relative.forearm', 
        'hand_w_rel':    'skeletal.weight.relative.hand', 
        'body_w':        'skeletal.weight.body', 
        'body_h':        'skeletal.size.body',
        'humerus_factor':'skeletal.size.relative.humerus_factor', 
        'lrte_b':        'muscles.biceps.tendon_length', 
        'lrte_t':        'muscles.triceps.tendon_length', 
        'g':             'skeletal.g', 
        'db':            'muscles.biceps.insertion', 
        'dt':            'muscles.triceps.insertion', 
    })
    
    def __init__(self, param_keys): 
        pmap = OrderedDict(ElbowModel.param_map)
        
        param_maps = dict()
        for muscle in ('biceps', 'triceps'): 
            param_maps[muscle] = {k: f'muscles.{muscle}.hill_muscle.{k}' 
                  for k in HillMuscle.param_names if f'muscles.{muscle}.hill_muscle.{k}' in param_keys}
            param_maps[muscle].update(
                {k: f'afferents.Ia.{k}' for k in HillMuscle.param_names if f'afferents.Ia.{k}' in param_keys})
            param_maps[muscle].update(
                {k: f'afferents.Ib.{k}' for k in HillMuscle.param_names if f'afferents.Ib.{k}' in param_keys})
        
        self.biceps_model  = HillMuscle(param_map=param_maps['biceps'], state_keys=HillMuscle.state_names, input_keys=HillMuscle.input_names, param_keys=param_keys)
        self.triceps_model = HillMuscle(param_map=param_maps['triceps'], state_keys=HillMuscle.state_names, input_keys=HillMuscle.input_names, param_keys=param_keys)
        
        pmap.update({f'biceps.{k}': v for k, v in self.biceps_model.param_map.items()})
        pmap.update({f'triceps.{k}': v for k, v in self.triceps_model.param_map.items()})
        
        pnames = (*pmap.keys(),)
        
        super().__init__(ElbowModel.state_names, ElbowModel.input_names, pnames, param_map=pmap, 
                        state_keys=ElbowModel.state_names, input_keys=ElbowModel.input_names, param_keys=param_keys)
        
        self.n  = len(self.state_names)
        self.m  = len(self.input_names)
        self.nP = len(self.param_names)
    
    @staticmethod
    def body_mensurations(uarm_l_rel, farm_l_rel, hand_l_rel, uarm_w_rel, farm_w_rel, hand_w_rel, 
                          body_w, body_h, humerus_factor):
        uarm_l, farm_l, hand_l = (rel * body_h for rel in (uarm_l_rel, farm_l_rel, hand_l_rel))
        uarm_w, farm_w, hand_w = (rel * body_w for rel in (uarm_w_rel, farm_w_rel, hand_w_rel))
        hl = uarm_l * humerus_factor
        return uarm_l, farm_l, hand_l, uarm_w, farm_w, hand_w, hl
    
    @staticmethod
    def muscle_length(q, hl, db, dt, lrte_b, lrte_t): 
        lb = sympy.sqrt((db * sympy.sin(q))**2 + (hl - db * sympy.cos(q))**2) - lrte_b
        lt = sympy.sqrt((dt * sympy.sin(q))**2 + (hl + dt * sympy.cos(q))**2) - lrte_t
        return lb, lt
    
    @staticmethod
    def moment_arm(q, lb, lt, hl, db, dt, lrte_b, lrte_t):
        rb = hl * db * sympy.sin(q) / (lb + lrte_b)
#         rb = sympy.Max(rb, 1e-2) 
        rt = - hl * dt * sympy.sin(q) / (lt + lrte_t)
#         rt = sympy.Min(rt, -1e-2) 
        return rb, rt
    
    @staticmethod
    def elbow_dynamic_properties(farm_w, farm_l, hand_w, hand_l):
        farm_com = farm_l / 2.
        hand_com = (farm_l + hand_l/2)
        resultant_mass = farm_w * farm_com + hand_w * hand_com
        inertia        = farm_w * farm_com**2 + hand_w * hand_com**2
        return resultant_mass, inertia
    
    def eval_muscle_models(self, dq, aMNb, aMNt, gMNb, gMNt, lb, lt, rb, rt, args_b, args_t, **kwargs): 
        dlb = rb * dq
        dlt = rt * dq
        
        bic = self.biceps_model.compute_output((lb, dlb), (aMNb, gMNb), args_b, **kwargs)
        tri = self.triceps_model.compute_output((lt, dlt), (aMNt, gMNt), args_t, **kwargs)
        
        return bic, tri
    
    @staticmethod
    def joint_acceleration(q, q2, Fbiceps, Ftriceps, rb, rt, farm_w, farm_l, hand_w, hand_l, g): 
        resultant_mass, inertia = ElbowModel.elbow_dynamic_properties(farm_w, farm_l, hand_w, hand_l)
        Mmuscles = - rb * Fbiceps - rt * Ftriceps
        Mgravity = - resultant_mass * g * sympy.sin(q2 - q)
        
        Mjoint   = Mmuscles + Mgravity
        ddq = Mjoint / inertia
        return ddq
    
    @NamedParamModel.unvec_args
    def compute_dynamics(self, q, dq, q2, aMNb, aMNt, gMNb, gMNt, uarm_l_rel, farm_l_rel, hand_l_rel, 
                         uarm_w_rel, farm_w_rel, hand_w_rel, body_w, body_h, humerus_factor, lrte_b, lrte_t, g, db, dt, *args): 
        args_b = args[:len(args)//2]
        args_t = args[len(args)//2:]
        
        uarm_l, farm_l, hand_l, uarm_w, farm_w, hand_w, hl =\
            ElbowModel.body_mensurations(uarm_l_rel, farm_l_rel, hand_l_rel, uarm_w_rel, farm_w_rel, 
                                         hand_w_rel, body_w, body_h, humerus_factor)
        lb, lt = ElbowModel.muscle_length(q, hl, db, dt, lrte_b, lrte_t)
        rb, rt = ElbowModel.moment_arm(q, lb, lt, hl, db, dt, lrte_b, lrte_t)
        
        Fbiceps, Ftriceps = self.eval_muscle_models(
            dq, aMNb, aMNt, gMNb, gMNt, lb, lt, rb, rt, args_b, args_t, return_rates=False)
        
        ddq    = ElbowModel.joint_acceleration(q, q2, Fbiceps, Ftriceps, rb, rt, farm_w, farm_l, hand_w, hand_l, g)
        
        y = np.array([dq, ddq]).reshape((2, 1)) * 1e-2
        y = sympy.Matrix(y)
        return y
    
    
    @NamedParamModel.unvec_args
    def compute_output(self,  q, dq, q2, aMNb, aMNt, gMNb, gMNt, uarm_l_rel, farm_l_rel, hand_l_rel, 
                         uarm_w_rel, farm_w_rel, hand_w_rel, body_w, body_h, humerus_factor, lrte_b, lrte_t, g, db, dt, *args): 
        args_b = args[:len(args)//2]
        args_t = args[len(args)//2:]
        
        uarm_l, farm_l, hand_l, uarm_w, farm_w, hand_w, hl =\
            ElbowModel.body_mensurations(uarm_l_rel, farm_l_rel, hand_l_rel, uarm_w_rel, farm_w_rel, 
                                         hand_w_rel, body_w, body_h, humerus_factor)
        
        lb, lt = ElbowModel.muscle_length(q, hl, db, dt, lrte_b, lrte_t)
        rb, rt = ElbowModel.moment_arm(q, lb, lt, hl, db, dt, lrte_b, lrte_t)
        
        (rateIa_b, rateIb_b), (rateIa_t, rateIb_t) = \
            self.eval_muscle_models(dq, aMNb, aMNt, gMNb, gMNt, lb, lt, rb, rt, args_b, args_t, return_force=False)
        
        y = sympy.Matrix([[q], [aMNb], [aMNt], [gMNb], [gMNt], [rateIa_b], [rateIb_b], [rateIa_t], [rateIb_t]])
        return y

In [12]:
elbow = ElbowModel([*params.keys()])
elbow.ix, elbow.iv, elbow.ip; 

In [48]:
X = sympy.MatrixSymbol('x', elbow.n, 1)
V = sympy.MatrixSymbol('v', elbow.m, 1)
P = sympy.MatrixSymbol('p', len(param_keys), 1)

x = np.asarray(X)
v = np.asarray(V)
p = np.asarray(P)

expr1 = elbow.compute_dynamics(x, v, p);
elbow.compute_output(x, v, p);

In [92]:
expr = elbow.compute_output(x, v, p)


In [81]:
J0 = expr.jacobian(P)

In [83]:
J0 = J0.as_immutable()

In [ ]:
J0.as_explicit()

In [ ]:
compute_sym_df_d2f(func

In [95]:
from sympy.tensor.array import derive_by_array

J = derive_by_array(expr, X)

In [101]:
%timeit H = derive_by_array(derive_by_array(expr, P), P)

1.16 s ± 9.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [102]:
%timeit sympy.lambdify((X, V, P), H)

585 ms ± 23.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [85]:
print(sympy.Matrix(expr.jacobian(P).as_immutable().as_explicit()).jacobian(P))

TypeError: ``self`` must be a row or a column matrix

In [73]:
%timeit sympy.lambdify((X, V, P), expr.jacobian(P))
%timeit sympy.lambdify((X, V, P), sympy.cse(expr.jacobian(P), list=False)[1])
%timeit sympy.lambdify((X, V, P), sympy.cse(expr, list=False)[1].jacobian(p))

4.43 s ± 25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.6 s ± 7.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
136 ms ± 1.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [69]:
_x = np.random.randn(*x.shape)
_v = np.random.randn(*v.shape)
_p = np.random.randn(*p.shape)

In [72]:
J0 =  sympy.lambdify((X, V, P), expr.jacobian(P))
J1 = sympy.lambdify((X, V, P), sympy.cse(expr.jacobian(P), list=False)[1])
J2 = sympy.lambdify((X, V, P), sympy.cse(expr, list=False)[1].jacobian(p))

In [71]:
print(J0(_x, _v, _p))
# print(J1(_x, _v, _p))
print(J2(_x, _v, _p))

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 1.62636957e+01  1.13919422e+01  0.00000000e+00  4.21284014e-01
  -6.30908594e+01  2.04770277e+03  1.20132859e+01  6.76024394e+01
   2.87344930e+01  0.00000000e+00  0.00000000e+00 -1.27496689e+00
  -3.92254327e-01 -3.81106835e-01 -1.35237400e-05 -0.0000

TypeError: cannot determine truth value of Relational

NameError: name 'J2' is not defined

In [ ]:
import dill
dill.settings['recurse'] = True
save_deriv=True
load_deriv=False
if load_deriv: 
    for _ in ('df', 'd2f', 'dg', 'd2g'): 
        models[0][_] = dill.load(open(f'.musculoskeletal_{_}_ser', 'rb'))
if save_deriv: 
    for _ in ('df', 'd2f', 'dg', 'd2g'):
        dill.dump(models[0][_], open(f'.musculoskeletal_{_k}_ser', 'rb'))

In [15]:
pE, pC, constraints = [[params[k][i] for k in param_keys] for i in range(3)] 
pE = np.asarray(pE)
pC = np.asarray(pC)

models = [
    GaussianModel(
        f=elbow.compute_dynamics, 
        g=elbow.compute_output,
        n=elbow.n, m=elbow.m, 
        pE=pE, pC=pC, V=np.array([np.exp(-8)] * 5), W=np.array([np.exp(-8)] * elbow.n)
    ), 
    GaussianModel(l=elbow.m, V=np.array([np.exp(-4)] * elbow.m))
]

hgm = HierarchicalGaussianModel(*models)

Compiling derivatives, it might take some time... 
  Compiling f... f() ok.
  Compiling g... 

KeyboardInterrupt: 

In [ ]:
results = DEMInversion(hgm, states_embedding_order=2).generate(100)

In [ ]:
px.line(y=[*results.v[:, 0, :, 0].T])

In [19]:
J00 = sympy.lambdify([X, V, P], j0[1], 'numpy', cse=True)
J10 = sympy.lambdify([X, V, P], j1[0], 'numpy', cse=True)

NameError: name 'j0' is not defined

In [ ]:
def jacobians(expr, *args): 
    subs, simplexpr = sympy.cse(expr, list=False, optimizations='basic')
    symb, transf = ([s[i] for s in subs] for i in range(2))
    
    mtransf = [sympy.Matrix([[tr]]) for tr in transf]
    nvar   = sympy.Matrix(symb)

    dfds = simplexpr.jacobian(nvar)

    J = []
    for var in args:
        sprev = []
        Jtr_vecs = []
        for s, tr in zip(symb, mtransf): 
            dsdx = tr.jacobian(var)
        
            if len(sprev) > 0: 
                _sp = sympy.SparseMatrix(np.stack([sprev]))
                dsdsp = tr.jacobian(_sp)
                dspdx = sympy.SparseMatrix(np.concatenate(Jtr_vecs))
                dsdx += dsdsp @ dspdx
#                 print(dsdx.jacobian(var))
                
            sprev.append(s)
            Jtr_vecs.append(dsdx)

        dsdx = sympy.SparseMatrix(np.concatenate(Jtr_vecs))
        dfdx = dfds @ dsdx
        for s, tr in zip(symb, transf): 
            dfds.subs(s, tr)
        J.append(dfdx)
        
    return J

j1 = jacobians(elbow.compute_dynamics(x, v, p), X, V, P)
print(j1[0])
# for new, expr in subs:
#     var = 

In [ ]:
print(j0[0][0, 0])

In [ ]:
gen = DEMInversion(hgm).generate(50)